In [ ]:

import glob

import rasterio
from shapely.geometry import LineString

import utils.visualization as viz
from utils.blobifier import blobify_raster_file
from utils.vectorizer.vector_builder import VectorBuilder

from simplification.cutil import simplify_coords

! export PYTHONOPTIMIZE="" # show asserts










In [ ]:
input_filepath = glob.glob('data/sources/*.tif')[0]
output_filepath = 'data/sources/cleaned.tif'

cleaned = blobify_raster_file(input_filepath, output_filepath)

with rasterio.open(input_filepath) as src:
    meta = src.meta
    input_raster = src.read(1)

with rasterio.open(output_filepath) as src:
    meta = src.meta
    output_raster = src.read(1)

config = viz.get_show_config(input_raster)
viz.show_raster(input_raster, *config)
viz.show_raster(output_raster, *config)

In [ ]:
input_filepath = "data/sources/cleaned.tif"
output_filepath = "data/sources/smoothed.shp"

tolerance = 30 # 30 meters, same as maximum resolution of image

def simplify(segment):
    simplified = simplify_coords(segment.coords, tolerance)
    return LineString(simplified)

vector_builder = VectorBuilder(input_filepath)
vector_builder.run_per_segment(simplify)
vector_builder.rebuild()
vector_builder.save(output_filepath)

data = vector_builder.data
vectors = vector_builder.covers
viz.show_raster(data, *viz.get_show_config(data))
labels = [v.label for v in vectors]
cmap = viz.generate_color_map(labels)
modified_polygons = [v.modified_polygon for v in vectors]
viz.show_polygons(modified_polygons, labels, color_map=cmap)

